In [ ]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark= SparkSession.builder.master("local[4]").appName("Excercise-1").config('spark.sql.shuffle.partitions', 4).getOrCreate()


In [ ]:

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 4 --topic orders 
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  orders   --from-beginning

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 4 --topic statewise_earning 
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  statewise_earning   --from-beginning

In [ ]:
readfromkafkaDf= spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "orders").option("group-id","ordersEx1-nav").load()

In [ ]:
readfromkafkaDf.printSchema()

In [ ]:
ordersDf= readfromkafkaDf.selectExpr(("CAST(value as STRING)"),("timestamp"))
ordersDf.printSchema()

In [ ]:
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType, IntegerType
schema= StructType([
    StructField("order_id",LongType(),True),
    StructField("item_id", StringType(), True),
     StructField("price", IntegerType(), True),
    StructField("Qty", IntegerType(),  True),
    StructField("state", StringType(),  True)
    ])

In [ ]:
import pyspark.sql.functions as F
stringToJsonOrdersDf= ordersDf.withColumn("value", F.from_json("value",schema))
stringToJsonOrdersDf.printSchema()

In [ ]:
valueDf= stringToJsonOrdersDf.select("value.*","timestamp")
valueDf.printSchema()
  

In [ ]:
from pyspark.sql.functions import *

amountDf= valueDf.withColumn("Amount", (col("price") * col("qty")))


amountDf.printSchema()                        


In [ ]:
sumDf= amountDf.withWatermark("timestamp", "30 seconds").groupBy("state", F.window("timestamp", "1 minutes"))\
                                .agg( F.sum( col("Amount")).alias("Amount"))\
                                .drop("window")   
sumDf.printSchema()      
# echoOnconsole = sumDf\
#                 .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start()
                              

In [ ]:
sumDfTojsonDf= sumDf.selectExpr("to_json(struct(*)) AS value")
sumDfTojsonDf.printSchema()



In [ ]:
sumDfTojsonDf.writeStream\
             .format("kafka")\
            .outputMode("update")\
             .option("kafka.bootstrap.servers", "localhost:9092")\
            .option("topic", "statewise_earning")\
            .option("checkpointLocation", "file:///tmp/spark95")\
            .start()




In [ ]:
def processBatchData(candleBatchDf, batch_id):
    print ("process batch called", batch_id, "writing ", candleBatchDf.count())
    # result = candleBatchDf.collect()
    # print(result)
    # candleBatchDf.show(2)
    # write to csv/json/parqeut/database /jdbc etc
    # window is nested column, has its children columns called start and end
    # start and end are part of nested window column
    # append mode will not delete existin data, instead append to existing table
    (
     candleBatchDf
        .write
        .mode("append")
        .format("jdbc")
        .option("url", "jdbc:mysql://localhost:3306/stockdb?allowPublicKeyRetrieval=true&useSSL=false")
        .option("driver", "com.mysql.jdbc.Driver")
        .option("user", "team")
        .option("password", "Team1234!")
        .option("dbtable", "orders")
         .save()
    )
    
sumDf.writeStream.outputMode("append")\
 .option("checkpointLocation", "file:///tmp/spark33")\
 .foreachBatch(processBatchData).start()


